In [1]:
import numpy as np
import pandas as pd
import os
import datetime
import sys
import tqdm


In [7]:
mimic_path = '/ssd1/mimic3' # please use your mimic3 folder
save_path = '../data'

In [3]:
using_feature = [ 'ICUSTAYS', 'CHARTEVENTS','LABEVENTS']
data_dict ={}
for file in using_feature:
    temp_name = file +'.csv'
    
    data_dict[file.lower()] = pd.read_csv(mimic_path+'/'+temp_name, low_memory=False)

In [4]:
my_data_icu=data_dict['icustays']
my_data_icu['INTIME'] = pd.to_datetime(my_data_icu['INTIME'])
my_data_icu['OUTTIME'] = pd.to_datetime(my_data_icu['OUTTIME'])

In [5]:
my_data_chart=data_dict['chartevents']
my_data_chart=my_data_chart[my_data_chart['VALUENUM'].isna()==False]
my_data_chart=my_data_chart[my_data_chart['ERROR']!=1]
my_data_chart['CHARTTIME']=pd.to_datetime(my_data_chart['CHARTTIME'])

my_data_lab=data_dict['labevents']
my_data_lab=my_data_lab[my_data_lab['VALUENUM']>=0]
my_data_lab['CHARTTIME'] = pd.to_datetime(my_data_lab['CHARTTIME'])


lab_and_icu = my_data_lab.merge(my_data_icu[['HADM_ID','ICUSTAY_ID','INTIME','OUTTIME','LOS']],'left',on='HADM_ID')
start_time = pd.Timedelta(hours=6)
lab_and_icu_intime = lab_and_icu[(lab_and_icu['CHARTTIME'] >= (lab_and_icu['INTIME'] - start_time)) & (lab_and_icu['CHARTTIME'] <= lab_and_icu['OUTTIME'])]


my_data_chart=my_data_chart[['SUBJECT_ID','HADM_ID','ITEMID','CHARTTIME','VALUE','VALUEUOM']]
chart_and_icu= my_data_chart.merge(my_data_icu[['HADM_ID','ICUSTAY_ID','INTIME','OUTTIME','LOS']],'left',on='HADM_ID')

chart_and_icu_intime =chart_and_icu[(chart_and_icu['CHARTTIME'] >= chart_and_icu['INTIME']) & (chart_and_icu['CHARTTIME'] <= chart_and_icu['OUTTIME'])]
len(lab_and_icu_intime), len(lab_and_icu), len(chart_and_icu_intime) , len(chart_and_icu)

chart_lab_icu=pd.concat([chart_and_icu_intime,lab_and_icu_intime[['SUBJECT_ID','HADM_ID','ITEMID','CHARTTIME','VALUE','VALUEUOM','ICUSTAY_ID','INTIME','OUTTIME','LOS']]])

/home/sanghoon/.conda/envs/rdkit_env/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
chart_lab_icu.to_pickle(save_path+'/chart_lab_icu.pkl')

In [9]:
len(chart_lab_icu)

158482934